In [7]:
tags = TAGS.Document(path="../../datasets/tags-tsv/boylesque/TAGS - boylesque - Archive.tsv", suppress_warnings=True)
ids = tags.ids
tweets = TAGS.TweetSet(ids, suppress_warnings=True, progressbar=False, filter_key='full_text', filter_value="boylesque", include_retweets=False)

AttributeError: module 'TAGS' has no attribute 'TweetSet'

In [8]:
ids[0:10]

[957183619386179584,
 923051501399965696,
 982681455909273600,
 853943528912715777,
 837996413460021248,
 858807036829630464,
 1125792856709398534,
 1123995654747979782,
 820247487713640448,
 829977424716759040]

In [24]:
# Set up standard variables
errors, all_data, all_errors = 0, [], []
slices = tweets.tweets[0:200]

with open('./configuration/block_list.txt', 'r') as f:
    block_list = f.read().split("\n")

In [44]:
all_data = []
for i, tweet in enumerate(slices):
    clean_text = CleanText(tweet.full_text)
    clear_output(wait=True)
    print(f"{i}/{len(slices)} ({len(slices)-i} remaining) - errors: {errors}")
    print("\n- ".join(all_errors))
    inferred = False
    lat, lng, boundingbox, geolocator = None, None, None, None

    # Geolocation
    if tweet.geo:
        lat = tweet.geo['coordinates'][0]
        lng = tweet.geo['coordinates'][1]
        inferred = False
        geolocator = None
    elif tweet.user.location is not None: # and only_roman_chars(tweet.user.location)
        try:
            location = Location(
                tweet.user.location, 
                block=block_list, 
                replacements_yaml = "./configuration/location_mapping.yml"
            )
        except GeocoderTimedOut:
            location = None
            errors += 1
            cleaner = CleanText.CleanText(tweet.user.location)
            cleaner.digits = False
            cleaner.clean()
            all_errors.append(f"{i}: {cleaner.text}")
            if errors == 10: raise RuntimeError("Too many errors.") from None
        if location is not None and not location.error:
            if location.data is not None:
                lat = location.data.get('lat', None)
                lng = location.data.get('lng', None)
                boundingbox = location.data.get('boundingbox', None)
                inferred = True
                geolocator = location.geolocator
    elif tweet.user.location is not None and not only_roman_chars(tweet.user.location):
        pass # or:
                # print(f"Location contained non-roman characters: {tweet.user.location}")
    data = {
        'clean_text': clean_text,
        'boundingbox': boundingbox,
        'lat': lat,
        'lng': lng,
        'inferred': inferred,
        'date': time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet.created_at,'%a %b %d %H:%M:%S +0000 %Y')),
        'lang': tweet.lang,
        'geolocator': geolocator,
        'full_text': tweet.full_text
    }
    all_data.append(data)

199/200 (1 remaining) - errors: 0



In [45]:
df = pd.DataFrame.from_dict(all_data)

In [46]:
df

,clean_text,boundingbox,lat,lng,inferred,date,lang,geolocator,full_text
0,video game burlesque thurs march feat striptea...,"[51.7657055, 51.9771532, 0.6993788, 1.0268034]",51.8896903,0.8994651,True,2018-01-27 09:28:53,en,Nominatim,Video Game Burlesque! Thurs 29th March @TheBir...
1,macho rock preening faggotry collide part boyl...,"[33.7036216, 34.337306, -118.6681776, -118.155...",34.0536909,-118.2427666,True,2017-10-25 05:00:02,en,Nominatim,Macho ‘80s rock and preening faggotry collide ...
2,award winning international boylesque stars br...,None,39.7481,-104.996,False,2018-04-07 18:08:11,en,None,Award-winning international boylesque stars br...
3,back year,"[49.674, 61.061, -14.015517, 2.0919117]",54.7023545,-3.2765753,True,2017-03-04 12:01:28,en,Nominatim,Back for a 2nd year! #bhof #bhof17 #lousafire ...
4,mixing lineup dynamic dance boylesque punky my...,"[42.3730513, 42.4181407, -71.1345919, -71.0734...",42.3875968,-71.0994968,True,2019-05-07 16:01:47,en,Nominatim,Mixing up the lineup with dynamic dance and bo...
...,...,...,...,...,...,...,...,...,...
195,check stonewall inn rt hobby fun evening stone...,"[40.477399, 40.9161785, -74.25909, -73.7001809]",40.7127281,-74.0060152,True,2017-03-12 02:59:55,en,Nominatim,Check out Stonewall Inn: https://t.co/ACtUtlRe...
196,world boylesque photo credit mo pitz hidden wo...,None,None,None,False,2015-05-29 18:58:03,en,None,The World of “Boylesque”: \nPhoto Credit: Mo P...
197,man walks metal bar wearing hat know not afrai...,"[30.0984576, 30.5166255, -97.9367663, -97.5605...",30.2711286,-97.7436995,True,2019-07-28 01:25:27,en,Nominatim,"Man walks into a metal bar wearing that hat, y..."
198,,"[50.0099945, 50.0899945, 10.193302, 10.273302]",50.0499945,10.233302,True,2015-12-31 08:07:58,und,Nominatim,#maizucker #berlin #burlesque #bbf2015 #burles...


In [47]:
all_data[195]

{'clean_text': check stonewall inn rt hobby fun evening stonewall,
 'boundingbox': ['40.477399', '40.9161785', '-74.25909', '-73.7001809'],
 'lat': '40.7127281',
 'lng': '-74.0060152',
 'inferred': True,
 'date': '2017-03-12 02:59:55',
 'lang': 'en',
 'geolocator': 'Nominatim',
 'full_text': 'Check out Stonewall Inn: https://t.co/ACtUtlRegs - RT @BryanKnight66 Hobby to the #nyceagle after a fun #boylesque evening @ Stonewall In...'}

In [371]:
from shapely.geometry import Point
from shapely.geometry import box

point = Point(0.5, 0.5)
b = box(-75.7854388,-75.6325211,4.3951997,4.5895768)
print(b.contains(point))

True


In [ ]:
(-75.7854388, -75.6325211),
(4.3951997, 4.5895768),

In [546]:
# Bounding box is: "south Latitude, north Latitude, west Longitude, east Longitude"
# according to https://wiki.openstreetmap.org/wiki/Bounding_Box
for file in Path('./locations/').glob('colchester.json'):
    with file.open() as f:
        data = json.load(f)
        if data['boundingbox'] is not None:
            min_lat = data['boundingbox'][0]
            max_lat = data['boundingbox'][1]
            min_lng = data['boundingbox'][2]
            max_lng = data['boundingbox'][3]
            print(file, data['display_name'])
            print(f"(min_lng,max_lng,min_lat,max_lat)")
            print(f"({min_lng},{max_lng},{min_lat},{max_lat})")
            print(data['lat'], data['lng'])
            print("")

locations/colchester.json Colchester, Essex, East of England, England, United Kingdom
(min_lng,max_lng,min_lat,max_lat)
(0.6993788,1.0268034,51.7657055,51.9771532)
51.8896903 0.8994651



In [655]:
def get_boundingbox(location = None):
    if not location: raise RuntimeError(f"Location must be set to not None.")
    location = clean_location(tweet.user.location)
    if len(location):
        cache_file = location_cache_directory / Path(location + ".json")
        if cache_file.is_file():
            geo_data = read_cache(cache_file)
            return_data = geo_data.get('boundingbox', None)
            if isinstance(return_data, list): 
                print("return_data is a list!")
                return_data = [float(x) for x in return_data] # returned as list of floats: [NS/latitude min, NS/latitude max; EW/longitude min; EW/longitude max]
            return(return_data)
        else:
            log(f"Cannot read cache file / file does not exist: {cache_file}")
            return(None)

def get_user_location(tweet):
    if tweet.user.location is not None:
        location = clean_location(tweet.user.location)
    else:
        return(False)
    
    if not location or len(location) == 0 or not only_roman_chars(location): # for now, this is the implementation
        return(False)
    else:
        cache_file = location_cache_directory / Path(location + ".json")
        if cache_file.is_file():
            geo_data = read_cache(cache_file)
            lat = geo_data.get('lat', None)
            lng = geo_data.get('lng', None)
            if lat: lat = float(lat)
            if lng: lat = float(lng)
            return([lat, lng])
        else:
            log(f"Cannot read cache file / file does not exist: {cache_file}", 1)
            return(False)

def get_native_tweet_location(tweet):
    if tweet.geo is not None and tweet.geo.get('coordinates', None):
        coordinates = tweet.geo.get('coordinates', None)
        if coordinates: coordinates = [float(x) for x in coordinates]
        return(coordinates)
    else:
        return(False)

def return_keywords_in_bounding_box(tweets, location = None, boundingbox = [], include_inferred = True, filter = True):
    all_keywords, total_processed_tweets = {}, 0
    if not location and not boundingbox: raise RuntimeError("Location or boundingbox must be provided.")
    if location and boundingbox: raise RuntimeError("Location or boundingbox must be provided, not both.")

    # We have been provided with location, let's extract boundingbox
    if location: boundingbox = get_boundingbox(location = location)
        
    # We have been provided with boundingbox, let's make sure datatypes are correct
    if boundingbox and isinstance(boundingbox, list): boundingbox = [float(x) for x in boundingbox]
    
    if filter:
        total_unfiltered_tweets = len(tweets)
        log(f"Filtering {total_unfiltered_tweets} tweets.", 1)
        tweets = [x for x in tweets 
                      if not TAGS.clean_text(x.full_text).startswith("rt") 
                      and "boylesque" in TAGS.clean_text(x.full_text)
                 ]
        total_num_tweets = len(tweets)
        log(f"=> {total_num_tweets} filtered tweets.", 1)
    
    bar = progressbar.ProgressBar(maxval=total_num_tweets).start()
    for i, tweet in enumerate(tweets):
        clear_output()
        bar.update(i)
        if get_native_tweet_location(tweet):
            lat, lng = get_native_tweet_location(tweet)[0], get_native_tweet_location(tweet)[1]
        elif include_inferred and get_user_location(tweet):
            lat, lng = get_user_location(tweet)[0], get_user_location(tweet)[1]
        else:
            lat, lng = None, None

        # check if lat + lng + boundingbox exists
        if lat and lng and boundingbox:
            # TODO: test for type not float....!
            if (boundingbox[1] > lat > boundingbox[0]) and (boundingbox[3] > lng > boundingbox[2]):
                full_text = tweet.full_text.replace("amp", "")
                keywords = TAGS.clean_text(full_text)

                for keyword in keywords.split(" "):
                    if keyword not in all_keywords: all_keywords[keyword] = 0
                    all_keywords[keyword] += 1
                total_processed_tweets += 1
    bar.finish()

    return({'total_unfiltered_tweets': total_unfiltered_tweets, 'total_num_tweets': total_num_tweets, 'total_processed_tweets': total_processed_tweets, 'all_keywords': all_keywords})

In [656]:
# return_keywords_in_bounding_box(tweets.tweets, location = "colchester") # retrieve all the keywords from colchester..
# return_keywords_in_bounding_box(tweets.tweets, location='Austin, TX') # = retrieves all keywords from Austin, TX
# you can also use include_inferred parameter to exclude those tweets where location is inferred through the user's fill-in location [although I am not sure that works just yet]

# [min_lat, max_lat, min_lng, max_lng]
# A couple of boundingboxes:
#  - UK: ['51.765706', '59.645540', '-11.601563', '1.026803']
#  - US: ['24.76' ,'49.21', '-128.232422', '-66.88']
#       east coast: ['25', '49.21', '-81', '-66.5']
#       west coast: ['31', '49.21', '-128', '-116']


# Example: Get all keywords from tweets from UK: return_keywords_in_bounding_box(tweets.tweets, boundingbox=['51.765706', '59.645540', '-11.601563', '1.026803'])
# return_keywords_in_bounding_box(tweets.tweets, boundingbox=['51.765706' ,'59.645540', '-128.232422', '1.026803'])

# return_keywords_in_bounding_box(tweets.tweets, boundingbox=['24.76' ,'49.21', '-128.232422', '-66.88']) # = retrieves all keywords from the U.S... not very precise
# return_keywords_in_bounding_box(tweets.tweets, boundingbox=['31', '49.21', '-128', '-116']) # = retrieves keywords from the west coast of U.S. 
return_keywords_in_bounding_box(tweets.tweets, boundingbox=['25', '49.21', '-81', '-66.5']) # = retrieves keywords from the east coast of U.S. 


TypeError: '>' not supported between instances of 'float' and 'str'